# 제3장 웹 스크래핑을 통한 데이터 수집

* 오래전에 공부한 내용을 ipynb형식으로 변경한 것임.

* Web scraping : 웹 페이지에 게시된 데이터를 가져오는 방법
* API 데이터 추출 : 웹 페이지가 제공하는 데이터를 가져오는 방법

### HTML
* id는 고유 값을 가지며, 일반적으로 문서에서 한 번만 사용됨.
* class는 여러번 사용됨.

### XML (eXtensivle Markup Language)
* XML은 데이터 교환이 목적이므로, 태그가 구조 정의를 위한 목적으로 사용된다. 개발자가 태그를 직접 정의하여 쓸 수도 있다.
* XML은 특정 운영체제나 웹 브라우저에 구속되지 않는다.

### JSON (JavaScript Object Notation)
* XML과 유사한 목적으로 만들어졌으나, XML보다 간편하고 쉽게 데이터 해석이 가능하다.
* JSON은 {key : value} 쌍이 차례대로 쌓여 있는 리스트 형태이다.

### Ajax
* 사용자와 서버가 정보를 주고 받는 방식 중 하나다.
* 사용자의 요청에 따라 서버가 전체 페이지가 아닌 일부분만 보여주는 방식이며, 사용자의 요청에 따라 일부분만이 업데이트 되는 특징을 가지고 있다.

### BeautifulSoup
* 파싱 : HTML, XML 형식의 웹 페이지 구조를 파악, 사용자가 원하는 자료 추출 과정을 말함.

|Function|Explanation|
|---|---|
|find_all()|일치 키워드 모두 반환|
|find()|일치 키워드 첫 번째 발견 반환|
|prettify()|HTML 트리 구조|
|get_text()|HTML 문자열 반환|
|title|title 태그 전체 반환|
|title.name|title 태그 이름 반환|
|title.string|title 태그 문자열 반환|
|p|p 태그 전체 요소 반환|
|p.string|p 태그 문자열 반환|
|p.[`class`]|p 태그 class 속성 값 반환|
|a|a 태그 전체 요소|
|get('href')|a 태그 url|
|contents|하부 단위 태그 전체 요소 반환|
|parent|상부 단위 태그 전체 요소 반환|

In [ ]:
# 국회 홈페이지에서 국회의원 이름, 코드, 사진 추출
# 본 풀이에선 photo라는 file 생성이 선결되어 있어야 함.

from bs4 import BeautifulSoup
import requests
import re
import csv
import os

# Congress man current page
url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300'

# Data requests
req = requests.get(url)
req.status_code
'''
101 : 데이터 처리 중
200 : 서버가 성공적으로 데이터 요청 처리
300 : 웹 페이지가 새로운 위치로 이동
403 : 서버가 권한 부족으로 요청 거부
404 : 해당 웹 페이지 찾을 수 없음
500 : 서버 오류
503 : 서버 오버 로드 혹은 서버 유지 관리를 위해 다운되었음
'''
req.text
html = req.content

# Data analysis
soup = BeautifulSoup(html, 'html.parser')
member_list = soup.select('.memberna_list dl dt a')
'''
<div class="memberna_list"> 하위에 있는 일반 태그 <dl> <dt> <a>에서
<a> 태그에 놓여 있는 모든 정보를 가져옴
'''

with open('member_list.csv', 'w') as f:
    # 저장할 파일을 쓸 수 있는 형태(w)로 열고, 객체 f를 생성
    csv_writer = csv.writer(f)

    for member in member_list:
        # names of congress men
        name = member.text

        # IDs of Congress
        id_href = member['href']

        pattern = re.search(r'\d+', id_href)
        # re.search(정규표현식, 탐색하는 대상)
        # re.search는 해당 식의 존재 여부에 따라 True or False를 반환함
        if pattern:
            mem_id = pattern.group(0)
        else:
            mem_id = None
        # pattern이 T면 mem_id에 넣고, F이면 None값을 넣으라는 뜻.

        # Pics of Congress
        pic = open('photo/{}.jpg'.format(mem_id),'wb')
        request_photo =\
            requests.get('http://www.assembly.go.kr/photo/{}.jpg'.format(mem_id)).content
        pic.write(request_photo)
        csv_writer.writerow([name, mem_id])
